# **ReadMe**

### 1. Change the variable 'n' (number of bit in input )

## 2. Change the variable 'original_circuit' (circuit matrix)


---



#**TASK (1) - GENERATE FAULT MATRIX**

# **`Initial Setups`**

## change the 'n' and 'original circuit'

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# GENERATE INPUT-------------------------------------------------------------------------------------------------------------------------
import math

n=3
original_circuit=[[2,1,1],[0,2,1],[0,1,2],[1,0,2],[0,2,1]]


# n = Number of Input Bit
# original_circuit = matrix (2-D ARRAY) of the Reversible Circuit
# (0=blank, 1=control, 2=target). There are 5 lists inside which denotes 5 gates as the circuit diagram.


# [[2,1,1],[0,2,1],[0,1,2],[1,0,2],[0,2,1]]           # Ham3#Design-1 Benchmark Circuit as a 2D-List
# [[0,0,2],[1,0,2],[0,2,1],[2,1,1],[1,1,2],[0,1,2]]   # https://www.revlib.org/doc/real/3_17_13.jpg
# [[1,1,0,2],[1,2,0,0],[0,1,1,2],[0,0,0,2],[0,1,2,0]]   # https://www.revlib.org/doc/real/rd32-v1_68.jpg


## Algorithm to find input set

In [ ]:
input=[]
for i in range(0,int(math.pow(2,n))):
  x=bin(i).replace("0b", "")    # Remove the first '0b' ( e.g -> [0b101 --> 101])
  while (len(x)<n):
    x='0'+x     # Make the all inputs n-bit long (e.g -> [0-->000, 1-->001, 10-->010, 11-->011]) means adding required '0's in the front
  x=list(x)
  input.append(x)
print("INPUT        |")
print("-------------")
for i in input:
  print(i)
print("")
print("-----------------------------------------------------")

INPUT        |
-------------
['0', '0', '0']
['0', '0', '1']
['0', '1', '0']
['0', '1', '1']
['1', '0', '0']
['1', '0', '1']
['1', '1', '0']
['1', '1', '1']

-----------------------------------------------------


# **Defined ```Functions``` to generate fault Matrix**

## 1. function to find control lines and target lines position

In [ ]:
# SEPARATE THE CONTROL LINE AND TARGET LINE POSITIONS-----------------------------------------------------------------------------------
def find_control_target_position(circuit,control_pos,target_pos):
  for j in circuit:
    temp_control_pos=[]
    temp_target_pos=[]
    for pos in range(len(j)):
      if j[pos]==1:   # Find the control positions
        temp_control_pos.append(pos)
      elif j[pos]==2:   # Find the target positions
        temp_target_pos.append(pos)
    control_pos.append(temp_control_pos)
    target_pos.append(temp_target_pos)

## 2. function to generate the fault matrix

In [ ]:
# ALGORITHM TO FIND OUTPUT AT EVERY PHASE------------------------------------------------------------------------------------------------------------
def find_fault_matrix(i,circuit,control_pos,target_pos,output_list):
  import copy
  temp_output=copy.deepcopy(input[i])     # Updating the output each time for 5 gate and 8 input (5*8=40)
  if(len(output_list)==i):
    temp_list=[]
    temp_list.append(copy.deepcopy(temp_output))
    output_list.append(temp_list)

  for j in range(len(circuit)):
    flag=0
    if(len(control_pos[j])!=0):   # Checking if there is any control line or not
      for element in control_pos[j]:
        if(temp_output[element]=='1'):    # Checking if all the control line is '1' or not. if yes, then increamenting the flag.
          flag=flag+1
      # print(f"length : {len(control_pos[j])}")
      # print(f"flag : {flag}")
      if(flag==len(control_pos[j])):    # Checking if (flag size == no. of all control line) of a gate
        if(temp_output[target_pos[j][0]]=='1'):
          temp_output[target_pos[j][0]]='0'   # If the control line condition satisfied, the flip the target line value to ['1'-->'0']
        elif(temp_output[target_pos[j][0]]=='0'):
          temp_output[target_pos[j][0]]='1'   # If the control line condition satisfied, the flip the target line value to ['0'-->'1']
    # output.append(copy.deepcopy(temp_output))   # deep copy as "Lists are manipulated in place" for details see next section (https://colab.research.google.com/drive/1X9niAf2dEMrpooPdsxznOboy1MSS76MU#scrollTo=nb98TOuRp2Sj)
    else:
      if(len(target_pos[j])!=0):    # if there is no control but have target line then perform only NOT operation on target input
        if(temp_output[target_pos[j][0]]=='1'):   # if there is no control line we simply perform NOT operation on the target input
          temp_output[target_pos[j][0]]='0'
        elif(temp_output[target_pos[j][0]]=='0'): # if there is no control line we simply perform NOT operation on the target input
          temp_output[target_pos[j][0]]='1'
  output_list[i].append(copy.deepcopy(temp_output))
  # print(" ")

## 3. function to generate the Total row and column fault sum

In [ ]:
# FIND TOTAL ROW-WISE FAULTY OUTPUT
def total_row_sum(output_list):
  length_of_row=len(output_list[0])
  for i in output_list:
    temp=i[1]
    count=0
    count_list=[]
    for j in range(2,length_of_row):
      if (temp!=i[j]):
        count+=1
    count_list.append(f"       {count}       ")
    i.append(count_list)


# FIND TOTAL COLUMN-WISE FAULTY OUTPUT
def total_column_sum(output_list):
  length_of_row=len(output_list[0])
  list1=[]
  list1.append("Total column fault ---------->")
  list1.append("----")
  for i in range(2,length_of_row):
    count=0
    temp_list1=[]
    for j in range(len(output_list)):
      if (output_list[j][1]!=output_list[j][i]):
        count+=1
    temp_list1.append(f"     {count}     ")
    list1.append(temp_list1)
  output_list.append(list1)

# **TASK (2) - FAULT LOCALIZATION AND GENERATE OPTIMAL TEST SET (OTS)**

# **Defined ```functions``` for Fault Localization and to generate Optimal Test Set (OTS)**



## 1. fucntion to separate the input output column

In [ ]:
# THIS VARIABLE STORES THE INPUT AND ORIGINAL OUTPUT TABLE AS AN 2-D MATRIX

def separate_input_output(original_output_list, actual_input_output_column):
  for i in range(0,len(original_output_list)-1):
    actual_input_output_column.append(original_output_list[i][0:2])

  # TO PRINT THE TABLE WITHOUT PANDAS LIBRARY
  # for i in actual_input_output_column:
  #   print(i)


## 2. function to separate the faults column

In [ ]:
# TRIMMED FAULTED TABLE

def separate_fault(original_output_list, original_output_list_fault_only):
  for i in range(0,len(original_output_list)-1):
    original_output_list_fault_only.append(original_output_list[i][2:len(original_output_list[0])-1])

  # TO PRINT THE FAULT TABLE WITHOUT PANDAS LIBRARY
  # for i in original_output_list_fault_only:
  #   print(i)

## 3. function to generate colored fault table

In [ ]:
def colored_fault_table(xMGF, actual_input_output_column, original_output_list_fault_only):
  print(f"\n\n   ------------------------------COLORED FAULT TABLE--------------------------------\n")
  for i in range(len(original_output_list_fault_only[0])):
    print(f"{' '*len(original_output_list_fault_only[0][0])*1}| {xMGF}-{i+1} |{' '*len(original_output_list_fault_only[0][0])*1}",end="   ")
  print("")

  for i in range(len(original_output_list_fault_only)):
    for j in range(len(original_output_list_fault_only[0])):
      temp_input=actual_input_output_column[i][1]
      if(temp_input!=original_output_list_fault_only[i][j]):
        print(f"\033[1;37;41m{original_output_list_fault_only[i][j]}", end="")
        print("\033[0;37;47m  ",end="")
      else:
        print(f"\033[0;37;42m{original_output_list_fault_only[i][j]}", end="")
        print("\033[0;37;47m  ",end="")
    print("")

## 4. function to find the maximum unique row sum position (deletable row)

In [ ]:
# arr1=actual_input_output_column_smgf / actual_input_output_column_pmgf
# arr2=original_output_list_fault_only_smgf / original_output_list_fault_only_pmgf
# arr3=available_inputs_smgf / available_inputs_pmgf
# arr4=available_gates_smgf / available_gates_pmgf

from collections import defaultdict
def find_maximum_unique_row_sum_position(arr1,arr2,arr3,arr4):
  row_sum_dict={}   # STORES THE UNIQUE ROW SUMS FOR EACH ROW IN KEY-VALUE PAIR
  for i in arr3:
    original_output=arr1[i][1]
    count=0
    counter_dict=defaultdict(int)   # STORES THE COUNT OF THE FAULTY OUTPUTS IN CURRENT ROW (HOW MANY TIMES A FAULTY OUTPUT COMES IN THE CURRENT ROW)
    for j in arr4:
      if(arr2[i][j]!=original_output):
        counter_dict["".join(arr2[i][j])]+=1
    # print(counter_dict)
    for k in counter_dict.keys():
      if(counter_dict[k]==1):       # ONLY TAKES THE FAULTY OUTPUT WHICH APPEARED JUST ONE(1) TIME (MEANS THIS FAULTY OUTPUT IS UNIQUE IN THE CURRENT ROW)
        count+=1
    row_sum_dict[i]=count
  print(f"unique row sums = {row_sum_dict}")
  max_value=0
  max_row=0                         # THE ROW WITH MAXIMUM UNIQUE FAULTY OUTPUT
  for i in row_sum_dict.keys():
    if(row_sum_dict[i]>max_value):
      max_value=row_sum_dict[i]
      max_row=i
  return max_row

## 5. function to find deletable columns based on maximum row position

In [ ]:
# arr1=actual_input_output_column_smgf / actual_input_output_column_pmgf
# arr2=original_output_list_fault_only_smgf / original_output_list_fault_only_pmgf
# arr3=available_inputs_smgf / available_inputs_pmgf
# arr4=available_gates_smgf / available_gates_pmgf
# max_row=deletable_row_smgf / deletable_row_smgf

def find_deletable_columns(arr1,arr2,arr3,arr4,max_row):
  del_cols=[]
  original_output=arr1[max_row][1]
  temp_list=[]
  counter_dict=defaultdict(int)     # STORES ALL UNIQUE OUTPUTS (WORKS AS A SET OF FAULTY OUTPUT)
  for i in arr4:
    if(arr2[max_row][i]!=original_output):
      counter_dict["".join(arr2[max_row][i])]+=1
  # print(counter_dict)
  for i in arr4:
    if(arr2[max_row][i]!=original_output):
      if(counter_dict["".join(arr2[max_row][i])]==1):     # IF THE FAULTY OUTPUT OF THE CURRENT ROW APPEARED JUST ONE(1) TIME, THEN MARKED THE COLUMN AS UNIQUE
        del_cols.append(i)
  return del_cols

## 6. function to update the tables (update available inputs and gates)

In [ ]:
# arr1=available_inputs_smgf / available_inputs_smgf
# arr2=available_gates_smgf / available_gates_smgf
# arr3=deletable_row_smgf / deletable_row_smgf
# arr4=deletable_columns_smgf / deletable_columns_smgf

def update_tables(arr1,arr2,arr3,arr4):
  # REMOVE THE ROW
  arr1.remove(arr3)

  # REMOVE THE COLUMNS
  for i in arr4:
    arr2.remove(i)

## 7. function to generate SMGF Fault Localization Table and Optimal Test Set (OTS)

In [ ]:
# arr1=actual_input_output_column
# arr2=original_output_list_fault_only
# arr3=deletable_row
# arr4=deletable_columns

def generate_smgf_fault_localization_table_and_ots(arr1,arr2,arr3,arr4):
  for i in arr4:
    temp_list1=[]
    temp_list1.append(arr1[arr3][0])      # APPEND THE INPUT
    temp_list1.append(f"GATE-{i+1}")      # APPEND THE FAULT LOCATION
    temp_list1.append(arr1[arr3][1])      # APPEND THE ORIGINAL OUTPUT
    temp_list1.append(arr2[arr3][i])      # APPEND THE FAULTY OUTPUT

    temp_str1=""
    temp_output=arr2[arr3][i]
    for j in range(len(arr2[arr3])):
      if(arr1[arr3][1]!=arr2[arr3][j] and j!=i):
        if(arr2[arr3][j]==temp_output):
          if(j in deleted_columns_smgf):
            temp_str1+=f"GATE-{j+1} -> "
    temp_str1+=f"GATE-{i+1}"

    temp_list1.append(temp_str1)
    fault_localization_table_smgf.append(temp_list1)
    # fault_localization_table.a(temp_list2)


    if (arr1[arr3][0] not in optimal_test_set_smgf):
      optimal_test_set_smgf.append(arr1[arr3][0])    # GENERATE THE OPTIMAL TEST SET

## 8. function to generate PMGF Fault Localization Table and Optimal Test Set (OTS)

In [ ]:
# arr1=actual_input_output_column
# arr2=original_output_list_fault_only
# arr3=deletable_row
# arr4=deletable_columns

def generate_pmgf_fault_localization_table_and_ots(arr1,arr2,arr3,arr4):
  for i in arr4:
    temp_list1=[]
    temp_list1.append(arr1[arr3][0])      # APPEND THE INPUT
    temp_list1.append(f"PMGF-{i+1}")      # APPEND PMGF NAME
    temp_list1.append(pmgf_name_dict[f"PMGF-{i+1}"])      # APPEND THE PMGF FAULT LOCATION
    temp_list1.append(arr1[arr3][1])      # APPEND THE ORIGINAL OUTPUT
    temp_list1.append(arr2[arr3][i])      # APPEND THE FAULTY OUTPUT

    temp_str1=""
    temp_output=arr2[arr3][i]
    for j in range(len(arr2[arr3])):
      if(arr1[arr3][1]!=arr2[arr3][j] and j!=i):
        if(arr2[arr3][j]==temp_output):
          if(j in deleted_columns_pmgf):
            temp_str1+=f"PMGF-{j+1} -> "
    temp_str1+=f"PMGF-{i+1}"

    temp_list1.append(temp_str1)
    fault_localization_table_pmgf.append(temp_list1)
    # fault_localization_table.a(temp_list2)


    if (arr1[arr3][0] not in optimal_test_set_pmgf):
      optimal_test_set_pmgf.append(arr1[arr3][0])    # GENERATE THE OPTIMAL TEST SET

# **SMGF (Single Misiing Gate Fault)**

# **```Driver Code``` to generate the SMGF Fault Matrix**

In [ ]:
import pandas as pd

# THE MATRIX STORED IN THIS VARIABLE AS A 2D LIST
original_output_list_smgf=[]

# ITERATE THE EACH ROW FOR EACH INPUT SET
for i in range(len(input)):
  # For Original output print
  original_control_pos_smgf=[]    # List of control positions of all gates
  original_target_pos_smgf=[]     # List of target positions of all gates

  # 1) FIND CONTROL AND TARGET POSITIONS OF THE ORIGINAL CIRCUIT
  find_control_target_position(original_circuit, original_control_pos_smgf, original_target_pos_smgf)

  # 2) GENERATE THE ORIGINAL FAULT MATRIX
  find_fault_matrix(i,original_circuit, original_control_pos_smgf, original_target_pos_smgf, original_output_list_smgf)
  # print(f"{original_output_list}")

  # For Faulty output print
  import copy
  # missing_gate=[0,0,0]    # SMGF MISSING GATE
  missing_gate=[0]*len(original_circuit[0])    # SMGF MISSING GATE
  for j in range(len(original_circuit)):
    smgf_circuit=copy.deepcopy(original_circuit)
    x=smgf_circuit.pop(j)   # POP EACH GATE
    smgf_circuit.insert(j,missing_gate)   # INSERT THE MISSING GATE TO CREATE SMGF FAULT

    faulty_control_pos_smgf=[]    # List of control positions of all gates
    faulty_target_pos_smgf=[]     # List of target positions of all gates

    # 1) FIND CONTROL AND TARGET POSITIONS OF THE ORIGINAL CIRCUIT
    find_control_target_position(smgf_circuit, faulty_control_pos_smgf, faulty_target_pos_smgf)

    # 2) GENERATE THE ORIGINAL FAULT MATRIX
    find_fault_matrix(i, smgf_circuit, faulty_control_pos_smgf, faulty_target_pos_smgf, original_output_list_smgf)

# 3) GENERATE TOTAL ROW & COLUMN SUM
total_row_sum(original_output_list_smgf)
total_column_sum(original_output_list_smgf)

## Generate matrix using ```Pandas``` library

In [ ]:
smgf_columns=["INPUT","OUTPUT"]
for col in range(len(original_circuit)):
  smgf_columns.append(f"SMGF-{col+1}")
smgf_columns.append("TOTAL ROW FAULT")

print("-------------------------------------MATRIX FOR SMGF---------------------------------")
df_original=pd.DataFrame(original_output_list_smgf, columns=smgf_columns)
df_original

-------------------------------------MATRIX FOR SMGF---------------------------------


,INPUT,OUTPUT,SMGF-1,SMGF-2,SMGF-3,SMGF-4,SMGF-5,TOTAL ROW FAULT
0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",[ 0 ]
1,"[0, 0, 1]","[0, 1, 0]","[0, 1, 0]","[0, 1, 1]","[0, 0, 1]","[0, 1, 0]","[0, 1, 0]",[ 2 ]
2,"[0, 1, 0]","[0, 0, 1]","[0, 0, 1]","[0, 0, 1]","[0, 1, 0]","[0, 0, 1]","[0, 1, 1]",[ 2 ]
3,"[0, 1, 1]","[1, 0, 0]","[0, 1, 1]","[1, 0, 1]","[1, 0, 0]","[1, 1, 1]","[1, 0, 0]",[ 3 ]
4,"[1, 0, 0]","[1, 1, 1]","[1, 1, 1]","[1, 1, 1]","[1, 1, 1]","[1, 0, 0]","[1, 0, 1]",[ 2 ]
5,"[1, 0, 1]","[1, 0, 1]","[1, 0, 1]","[1, 0, 0]","[1, 1, 0]","[1, 1, 0]","[1, 1, 1]",[ 4 ]
6,"[1, 1, 0]","[1, 1, 0]","[1, 1, 0]","[1, 1, 0]","[1, 0, 1]","[1, 0, 1]","[1, 1, 0]",[ 2 ]
7,"[1, 1, 1]","[0, 1, 1]","[1, 0, 0]","[0, 1, 0]","[0, 1, 1]","[0, 1, 1]","[0, 0, 1]",[ 3 ]
8,Total column fault ---------->,----,[ 2 ],[ 4 ],[ 4 ],[ 4 ],[ 4 ],[ 8 ]


# **```Driver Code``` to generate the SMGF Fault Fault Localization Table**

## separate input-output column for SMGF

In [ ]:
# 1) SEPARATE INPUT-OUTPUT COLUMN
actual_input_output_column_smgf=[]
separate_input_output(original_output_list_smgf, actual_input_output_column_smgf)

print("--INPUT-OUTPUT TABLE--")
input_columns_smgf=(["Input","Output"])
df_input = pd.DataFrame(actual_input_output_column_smgf,columns=input_columns_smgf)
df_input


--INPUT-OUTPUT TABLE--


,Input,Output
0,"[0, 0, 0]","[0, 0, 0]"
1,"[0, 0, 1]","[0, 1, 0]"
2,"[0, 1, 0]","[0, 0, 1]"
3,"[0, 1, 1]","[1, 0, 0]"
4,"[1, 0, 0]","[1, 1, 1]"
5,"[1, 0, 1]","[1, 0, 1]"
6,"[1, 1, 0]","[1, 1, 0]"
7,"[1, 1, 1]","[0, 1, 1]"


## separate faults column for SMGF

In [ ]:
# 2) SEPARATE FAULTS COLUMN
original_output_list_fault_only_smgf=[]
separate_fault(original_output_list_smgf, original_output_list_fault_only_smgf)

print(f"\n\n   -----------------------FAULT TABLE----------------------\n")
fault_columns_smgf=[]
for i in range(len(original_output_list_fault_only_smgf[0])):
  fault_columns_smgf.append(f"SMGF-{i+1}")

df_fault = pd.DataFrame(original_output_list_fault_only_smgf,columns=fault_columns_smgf)
df_fault



   -----------------------FAULT TABLE----------------------



,SMGF-1,SMGF-2,SMGF-3,SMGF-4,SMGF-5
0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]"
1,"[0, 1, 0]","[0, 1, 1]","[0, 0, 1]","[0, 1, 0]","[0, 1, 0]"
2,"[0, 0, 1]","[0, 0, 1]","[0, 1, 0]","[0, 0, 1]","[0, 1, 1]"
3,"[0, 1, 1]","[1, 0, 1]","[1, 0, 0]","[1, 1, 1]","[1, 0, 0]"
4,"[1, 1, 1]","[1, 1, 1]","[1, 1, 1]","[1, 0, 0]","[1, 0, 1]"
5,"[1, 0, 1]","[1, 0, 0]","[1, 1, 0]","[1, 1, 0]","[1, 1, 1]"
6,"[1, 1, 0]","[1, 1, 0]","[1, 0, 1]","[1, 0, 1]","[1, 1, 0]"
7,"[1, 0, 0]","[0, 1, 0]","[0, 1, 1]","[0, 1, 1]","[0, 0, 1]"


## Generate Colored fault table SMGF

In [ ]:
# 3) GENERATE COLORED FAULT TABLE SMGF
colored_fault_table("SMGF",actual_input_output_column_smgf, original_output_list_fault_only_smgf)



   ------------------------------COLORED FAULT TABLE--------------------------------

   | SMGF-1 |         | SMGF-2 |         | SMGF-3 |         | SMGF-4 |         | SMGF-5 |      
['0', '0', '0']  ['0', '0', '0']  ['0', '0', '0']  ['0', '0', '0']  ['0', '0', '0']  
['0', '1', '0']  ['0', '1', '1']  ['0', '0', '1']  ['0', '1', '0']  ['0', '1', '0']  
['0', '0', '1']  ['0', '0', '1']  ['0', '1', '0']  ['0', '0', '1']  ['0', '1', '1']  
['0', '1', '1']  ['1', '0', '1']  ['1', '0', '0']  ['1', '1', '1']  ['1', '0', '0']  
['1', '1', '1']  ['1', '1', '1']  ['1', '1', '1']  ['1', '0', '0']  ['1', '0', '1']  
['1', '0', '1']  ['1', '0', '0']  ['1', '1', '0']  ['1', '1', '0']  ['1', '1', '1']  
['1', '1', '0']  ['1', '1', '0']  ['1', '0', '1']  ['1', '0', '1']  ['1', '1', '0']  
['1', '0', '0']  ['0', '1', '0']  ['0', '1', '1']  ['0', '1', '1']  ['0', '0', '1']  


## **```Driver Code SMGF```**

In [ ]:
available_inputs_smgf=[i for i in range(0,len(original_output_list_fault_only_smgf))]       # GLOBAL VARIABLE, STORES THE AVAILABLE ROWS OF THE TABLE
available_gates_smgf=[i for i in range(0,len(original_output_list_fault_only_smgf[0]))]     # GLOBAL VARIABLE, STORES THE AVAILABLE COLUMNS OF THE TABLE
optimal_test_set_smgf=[]       # GLOBAL VARIABLE, STORES THE OTS
fault_localization_table_smgf=[]       # GLOBAL VARIABLE, STORES THE FAULT LOCALIZATION TABLE
deleted_columns_smgf=[]

iteration_smgf=1         # GLOBAL VARIABLE, STORES THE CURRENT ITERATION NUMBER

print(f"Total Inputs/Rows    = {n}-bit | {available_inputs_smgf}")
print(f"Total Gates/Columns  = {len(original_circuit)}-Gates | {available_gates_smgf}")
print("----------------------------------------------------------")

# DRIVER CODE------------------
while(len(available_gates_smgf)!=0):
  print(f"ITERATION-{iteration_smgf}")

  # 4) FIND DELETABLE ROW
  deletable_row_smgf=find_maximum_unique_row_sum_position(actual_input_output_column_smgf,original_output_list_fault_only_smgf,available_inputs_smgf,available_gates_smgf)
  print(f"deletable row = {deletable_row_smgf}")

  # 5) FIND DELETABLE COLUMNS
  deletable_columns_smgf=sorted(find_deletable_columns(actual_input_output_column_smgf,original_output_list_fault_only_smgf,available_inputs_smgf,available_gates_smgf,deletable_row_smgf))
  print(f"deletable columns = {deletable_columns_smgf}")
  deleted_columns_smgf.extend(deletable_columns_smgf)
  print(f"Already Deleted columns = {deleted_columns_smgf}")

  # 6) UPDATE THE INPUT-OUTPUT TABLE AND FAULT TABLE
  update_tables(available_inputs_smgf,available_gates_smgf,deletable_row_smgf,deletable_columns_smgf)
  print(f"available inputs/rows = {available_inputs_smgf}")
  if(len(available_gates_smgf)!=0):
    print(f"available gates/columns = {available_gates_smgf}")
  else:
    print("[ NO AVAILABLE GATEs / COLUMNs ]")

  # 7) GENERATE THE SMGF FAULT LOCALIZATION TABLE AND OTS
  generate_smgf_fault_localization_table_and_ots(actual_input_output_column_smgf,original_output_list_fault_only_smgf,deletable_row_smgf,deletable_columns_smgf)

  iteration_smgf+=1

  print("--------------------------------------------------")


# # for i in fault_localization_table:
# #   print(i)
# print(f"\n\033[1;37;42mOptimal Test Set = {optimal_test_set_smgf}\n")
# fault_table_columns_smgf=(["INPUT","FAULT_LOCATION","ORIGINAL_OUTPUT","FAULTY_OUTPUT","INCREASING ORDER"])
# df_fault_table=pd.DataFrame(fault_localization_table_smgf, columns=fault_table_columns_smgf)
# df_fault_table
# print("\033[1;37;41mNOTE : THE INCREASING ORDER OF THE FAULTS MUST BE FOLLOW FOR LOCALIZATION-------------------")



Total Inputs/Rows    = 3-bit | [0, 1, 2, 3, 4, 5, 6, 7]
Total Gates/Columns  = 5-Gates | [0, 1, 2, 3, 4]
----------------------------------------------------------
ITERATION-1
unique row sums = {0: 0, 1: 2, 2: 2, 3: 3, 4: 2, 5: 2, 6: 0, 7: 3}
deletable row = 3
deletable columns = [0, 1, 3]
Already Deleted columns = [0, 1, 3]
available inputs/rows = [0, 1, 2, 4, 5, 6, 7]
available gates/columns = [2, 4]
--------------------------------------------------
ITERATION-2
unique row sums = {0: 0, 1: 1, 2: 2, 4: 1, 5: 2, 6: 1, 7: 1}
deletable row = 2
deletable columns = [2, 4]
Already Deleted columns = [0, 1, 3, 2, 4]
available inputs/rows = [0, 1, 4, 5, 6, 7]
[ NO AVAILABLE GATEs / COLUMNs ]
--------------------------------------------------


In [ ]:
for i in original_output_list_fault_only_smgf:
  print(i)

[['0', '0', '0'], ['0', '0', '0'], ['0', '0', '0'], ['0', '0', '0'], ['0', '0', '0']]
[['0', '1', '0'], ['0', '1', '1'], ['0', '0', '1'], ['0', '1', '0'], ['0', '1', '0']]
[['0', '0', '1'], ['0', '0', '1'], ['0', '1', '0'], ['0', '0', '1'], ['0', '1', '1']]
[['0', '1', '1'], ['1', '0', '1'], ['1', '0', '0'], ['1', '1', '1'], ['1', '0', '0']]
[['1', '1', '1'], ['1', '1', '1'], ['1', '1', '1'], ['1', '0', '0'], ['1', '0', '1']]
[['1', '0', '1'], ['1', '0', '0'], ['1', '1', '0'], ['1', '1', '0'], ['1', '1', '1']]
[['1', '1', '0'], ['1', '1', '0'], ['1', '0', '1'], ['1', '0', '1'], ['1', '1', '0']]
[['1', '0', '0'], ['0', '1', '0'], ['0', '1', '1'], ['0', '1', '1'], ['0', '0', '1']]


# **PMGF (Partial Missing Gate Fault)**

# **convert the original circuit to `PMGF` circuit**

In [ ]:
pmgf_circuit=[]
pmgf_name_dict={}

import copy

pmgf_count=1
for i in range(len(original_circuit)):
  for j in range(len(original_circuit[0])):
    if(original_circuit[i][j]==1):
      temp_pmgf=copy.deepcopy(original_circuit)
      temp_pmgf[i][j]=0
      pmgf_circuit.append(temp_pmgf)
      pmgf_name_dict[f"PMGF-{pmgf_count}"]=f"GATE={i+1}  ||  INPUT-LINE={j+1}"
      pmgf_count+=1

print("PMGF-NAME      PMGF-CIRCUIT")
print("---------      -------------")
for i in range (len(pmgf_circuit)):
  print(f"PMGF-{i+1} ==>     {pmgf_circuit[i]}")

PMGF-NAME      PMGF-CIRCUIT
---------      -------------
PMGF-1 ==>     [[2, 0, 1], [0, 2, 1], [0, 1, 2], [1, 0, 2], [0, 2, 1]]
PMGF-2 ==>     [[2, 1, 0], [0, 2, 1], [0, 1, 2], [1, 0, 2], [0, 2, 1]]
PMGF-3 ==>     [[2, 1, 1], [0, 2, 0], [0, 1, 2], [1, 0, 2], [0, 2, 1]]
PMGF-4 ==>     [[2, 1, 1], [0, 2, 1], [0, 0, 2], [1, 0, 2], [0, 2, 1]]
PMGF-5 ==>     [[2, 1, 1], [0, 2, 1], [0, 1, 2], [0, 0, 2], [0, 2, 1]]
PMGF-6 ==>     [[2, 1, 1], [0, 2, 1], [0, 1, 2], [1, 0, 2], [0, 2, 0]]


# **`PMGF` name corresponds to fault location**

In [ ]:
print("PMGF-NAME      PMGF-LOCATION")
print("---------      -------------")
for name,location in pmgf_name_dict.items():
  print(f"{name} ==>     {location}")

PMGF-NAME      PMGF-LOCATION
---------      -------------
PMGF-1 ==>     GATE=1  ||  INPUT-LINE=2
PMGF-2 ==>     GATE=1  ||  INPUT-LINE=3
PMGF-3 ==>     GATE=2  ||  INPUT-LINE=3
PMGF-4 ==>     GATE=3  ||  INPUT-LINE=2
PMGF-5 ==>     GATE=4  ||  INPUT-LINE=1
PMGF-6 ==>     GATE=5  ||  INPUT-LINE=3


# **```Driver Code``` to generate the PMGF Fault Matrix**

In [ ]:
import pandas as pd

# THE MATRIX STORED IN THIS VARIABLE AS A 2D LIST
original_output_list_pmgf=[]

# ITERATE THE EACH ROW FOR EACH INPUT SET
for i in range(len(input)):
  # For Original output print
  original_control_pos_pmgf=[]    # List of control positions of all gates
  original_target_pos_pmgf=[]     # List of target positions of all gates

  # 1) FIND CONTROL AND TARGET POSITIONS OF THE ORIGINAL CIRCUIT
  find_control_target_position(original_circuit, original_control_pos_pmgf, original_target_pos_pmgf)

  # 2) GENERATE THE ORIGINAL FAULT MATRIX
  find_fault_matrix(i,original_circuit, original_control_pos_pmgf, original_target_pos_pmgf, original_output_list_pmgf)
  # print(f"{original_output_list}")

  # For PMGF Faulty output print
  import copy

  for j in range(len(pmgf_circuit)):

    faulty_control_pos_pmgf=[]    # List of control positions of all gates
    faulty_target_pos_pmgf=[]     # List of target positions of all gates

    # 1) FIND CONTROL AND TARGET POSITIONS OF THE ORIGINAL CIRCUIT
    find_control_target_position(pmgf_circuit[j], faulty_control_pos_pmgf, faulty_target_pos_pmgf)

    # 2) GENERATE THE ORIGINAL FAULT MATRIX
    find_fault_matrix(i, pmgf_circuit[j], faulty_control_pos_pmgf, faulty_target_pos_pmgf, original_output_list_pmgf)

# 3) GENERATE TOTAL ROW & COLUMN SUM
total_row_sum(original_output_list_pmgf)
total_column_sum(original_output_list_pmgf)

## Generate matrix using ```Pandas``` library

In [ ]:
pmgf_columns=["INPUT","OUTPUT"]
for col in range(len(pmgf_circuit)):
  pmgf_columns.append(f"PMGF-{col+1}")
pmgf_columns.append("TOTAL ROW FAULT")

print("-------------------------------------MATRIX FOR PMGF---------------------------------")
df_original=pd.DataFrame(original_output_list_pmgf, columns=pmgf_columns)
df_original

-------------------------------------MATRIX FOR PMGF---------------------------------


,INPUT,OUTPUT,PMGF-1,PMGF-2,PMGF-3,PMGF-4,PMGF-5,PMGF-6,TOTAL ROW FAULT
0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 1]","[0, 1, 1]","[0, 1, 1]","[0, 1, 0]",[ 4 ]
1,"[0, 0, 1]","[0, 1, 0]","[1, 0, 1]","[0, 1, 0]","[0, 1, 0]","[0, 1, 0]","[0, 0, 1]","[0, 0, 0]",[ 3 ]
2,"[0, 1, 0]","[0, 0, 1]","[0, 0, 1]","[1, 1, 0]","[0, 0, 0]","[0, 0, 1]","[0, 1, 0]","[0, 0, 1]",[ 3 ]
3,"[0, 1, 1]","[1, 0, 0]","[1, 0, 0]","[1, 0, 0]","[1, 0, 0]","[1, 1, 1]","[1, 0, 0]","[1, 1, 0]",[ 2 ]
4,"[1, 0, 0]","[1, 1, 1]","[1, 1, 1]","[1, 1, 1]","[1, 1, 0]","[1, 0, 0]","[1, 1, 1]","[1, 1, 1]",[ 2 ]
5,"[1, 0, 1]","[1, 0, 1]","[0, 1, 0]","[1, 0, 1]","[1, 0, 1]","[1, 0, 1]","[1, 0, 1]","[1, 0, 1]",[ 1 ]
6,"[1, 1, 0]","[1, 1, 0]","[1, 1, 0]","[0, 0, 1]","[1, 1, 1]","[1, 1, 0]","[1, 1, 0]","[1, 0, 0]",[ 3 ]
7,"[1, 1, 1]","[0, 1, 1]","[0, 1, 1]","[0, 1, 1]","[0, 1, 1]","[0, 0, 0]","[0, 0, 0]","[0, 1, 1]",[ 2 ]
8,Total column fault ---------->,----,[ 2 ],[ 2 ],[ 4 ],[ 4 ],[ 4 ],[ 4 ],[ 8 ]


# **```Driver Code``` to generate the Fault Fault Localization Table**

## separate input-output column for PMGF

In [ ]:
# 1) SEPARATE INPUT-OUTPUT COLUMN
actual_input_output_column_pmgf=[]
separate_input_output(original_output_list_pmgf, actual_input_output_column_pmgf)

print("--INPUT-OUTPUT TABLE--")
input_columns_pmgf=(["Input","Output"])
df_input = pd.DataFrame(actual_input_output_column_pmgf,columns=input_columns_pmgf)
df_input


--INPUT-OUTPUT TABLE--


,Input,Output
0,"[0, 0, 0]","[0, 0, 0]"
1,"[0, 0, 1]","[0, 1, 0]"
2,"[0, 1, 0]","[0, 0, 1]"
3,"[0, 1, 1]","[1, 0, 0]"
4,"[1, 0, 0]","[1, 1, 1]"
5,"[1, 0, 1]","[1, 0, 1]"
6,"[1, 1, 0]","[1, 1, 0]"
7,"[1, 1, 1]","[0, 1, 1]"


## separate faults column for PMGF

In [ ]:
# 2) SEPARATE FAULTS COLUMN
original_output_list_fault_only_pmgf=[]
separate_fault(original_output_list_pmgf, original_output_list_fault_only_pmgf)

print(f"\n\n   -----------------------FAULT TABLE----------------------\n")
fault_columns_pmgf=[]
for i in range(len(original_output_list_fault_only_pmgf[0])):
  fault_columns_pmgf.append(f"PMGF-{i+1}")

df_fault = pd.DataFrame(original_output_list_fault_only_pmgf,columns=fault_columns_pmgf)
df_fault



   -----------------------FAULT TABLE----------------------



,PMGF-1,PMGF-2,PMGF-3,PMGF-4,PMGF-5,PMGF-6
0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 1]","[0, 1, 1]","[0, 1, 1]","[0, 1, 0]"
1,"[1, 0, 1]","[0, 1, 0]","[0, 1, 0]","[0, 1, 0]","[0, 0, 1]","[0, 0, 0]"
2,"[0, 0, 1]","[1, 1, 0]","[0, 0, 0]","[0, 0, 1]","[0, 1, 0]","[0, 0, 1]"
3,"[1, 0, 0]","[1, 0, 0]","[1, 0, 0]","[1, 1, 1]","[1, 0, 0]","[1, 1, 0]"
4,"[1, 1, 1]","[1, 1, 1]","[1, 1, 0]","[1, 0, 0]","[1, 1, 1]","[1, 1, 1]"
5,"[0, 1, 0]","[1, 0, 1]","[1, 0, 1]","[1, 0, 1]","[1, 0, 1]","[1, 0, 1]"
6,"[1, 1, 0]","[0, 0, 1]","[1, 1, 1]","[1, 1, 0]","[1, 1, 0]","[1, 0, 0]"
7,"[0, 1, 1]","[0, 1, 1]","[0, 1, 1]","[0, 0, 0]","[0, 0, 0]","[0, 1, 1]"


## Generate Colored fault table PMGF

In [ ]:
# 3) GENERATE COLORED FAULT TABLE PMGF
colored_fault_table("PMGF", actual_input_output_column_pmgf, original_output_list_fault_only_pmgf)



   ------------------------------COLORED FAULT TABLE--------------------------------

   | PMGF-1 |         | PMGF-2 |         | PMGF-3 |         | PMGF-4 |         | PMGF-5 |         | PMGF-6 |      
['0', '0', '0']  ['0', '0', '0']  ['0', '0', '1']  ['0', '1', '1']  ['0', '1', '1']  ['0', '1', '0']  
['1', '0', '1']  ['0', '1', '0']  ['0', '1', '0']  ['0', '1', '0']  ['0', '0', '1']  ['0', '0', '0']  
['0', '0', '1']  ['1', '1', '0']  ['0', '0', '0']  ['0', '0', '1']  ['0', '1', '0']  ['0', '0', '1']  
['1', '0', '0']  ['1', '0', '0']  ['1', '0', '0']  ['1', '1', '1']  ['1', '0', '0']  ['1', '1', '0']  
['1', '1', '1']  ['1', '1', '1']  ['1', '1', '0']  ['1', '0', '0']  ['1', '1', '1']  ['1', '1', '1']  
['0', '1', '0']  ['1', '0', '1']  ['1', '0', '1']  ['1', '0', '1']  ['1', '0', '1']  ['1', '0', '1']  
['1', '1', '0']  ['0', '0', '1']  ['1', '1', '1']  ['1', '1', '0']  ['1', '1', '0']  ['1', '0', '0']  
['0', '1', '1']  ['0', '1', '1']  ['0', '1', '1']  ['0', '0', '0']  ['0', '0

## **```Driver Code PMGF```**

In [ ]:
available_inputs_pmgf=[i for i in range(0,len(original_output_list_fault_only_pmgf))]       # GLOBAL VARIABLE, STORES THE AVAILABLE ROWS OF THE TABLE
available_gates_pmgf=[i for i in range(0,len(original_output_list_fault_only_pmgf[0]))]     # GLOBAL VARIABLE, STORES THE AVAILABLE COLUMNS OF THE TABLE
optimal_test_set_pmgf=[]       # GLOBAL VARIABLE, STORES THE OTS
fault_localization_table_pmgf=[]       # GLOBAL VARIABLE, STORES THE FAULT LOCALIZATION TABLE
deleted_columns_pmgf=[]

iteration_pmgf=1         # GLOBAL VARIABLE, STORES THE CURRENT ITERATION NUMBER

print(f"Total Inputs/Rows    = {n}-bit | {available_inputs_pmgf}")
print(f"Total Gates/Columns  = {len(original_circuit)}-Gates | {available_gates_pmgf}")
print("----------------------------------------------------------")

# DRIVER CODE------------------
while(len(available_gates_pmgf)!=0):
  print(f"ITERATION-{iteration_pmgf}")

  # 4) FIND DELETABLE ROW
  deletable_row_pmgf=find_maximum_unique_row_sum_position(actual_input_output_column_pmgf,original_output_list_fault_only_pmgf,available_inputs_pmgf,available_gates_pmgf)
  print(f"deletable row = {deletable_row_pmgf}")

  # 5) FIND DELETABLE COLUMNS
  deletable_columns_pmgf=sorted(find_deletable_columns(actual_input_output_column_pmgf,original_output_list_fault_only_pmgf,available_inputs_pmgf,available_gates_pmgf,deletable_row_pmgf))
  print(f"deletable columns = {deletable_columns_pmgf}")
  deleted_columns_pmgf.extend(deletable_columns_pmgf)
  print(f"Already Deleted columns = {deleted_columns_pmgf}")

  # 6) UPDATE THE INPUT-OUTPUT TABLE AND FAULT TABLE
  update_tables(available_inputs_pmgf,available_gates_pmgf,deletable_row_pmgf,deletable_columns_pmgf)
  print(f"available inputs/rows = {available_inputs_pmgf}")
  if(len(available_gates_pmgf)!=0):
    print(f"available gates/columns = {available_gates_pmgf}")
  else:
    print("[ NO AVAILABLE GATEs / COLUMNs ]")

  # 8) GENERATE THE PMGF FAULT LOCALIZATION TABLE AND OTS
  generate_pmgf_fault_localization_table_and_ots(actual_input_output_column_pmgf,original_output_list_fault_only_pmgf,deletable_row_pmgf,deletable_columns_pmgf)

  iteration_pmgf+=1

  print("--------------------------------------------------")


# for i in fault_localization_table:
#   print(i)
# print(f"\n\033[1;37;42mOptimal Test Set = {optimal_test_set_pmgf}\n")
# fault_table_columns_pmgf=(["INPUT","PMGF_NAME","FAULT_LOCATION","ORIGINAL_OUTPUT","FAULTY_OUTPUT","INCREASING ORDER"])
# df_fault_table=pd.DataFrame(fault_localization_table_pmgf, columns=fault_table_columns_pmgf)
# df_fault_table
# print("\033[1;37;41mNOTE : THE INCREASING ORDER OF THE FAULTS MUST BE FOLLOW FOR LOCALIZATION-------------------")



Total Inputs/Rows    = 3-bit | [0, 1, 2, 3, 4, 5, 6, 7]
Total Gates/Columns  = 5-Gates | [0, 1, 2, 3, 4, 5]
----------------------------------------------------------
ITERATION-1
unique row sums = {0: 2, 1: 3, 2: 3, 3: 2, 4: 2, 5: 1, 6: 3, 7: 0}
deletable row = 1
deletable columns = [0, 4, 5]
Already Deleted columns = [0, 4, 5]
available inputs/rows = [0, 2, 3, 4, 5, 6, 7]
available gates/columns = [1, 2, 3]
--------------------------------------------------
ITERATION-2
unique row sums = {0: 2, 2: 2, 3: 1, 4: 2, 5: 0, 6: 2, 7: 1}
deletable row = 0
deletable columns = [2, 3]
Already Deleted columns = [0, 4, 5, 2, 3]
available inputs/rows = [2, 3, 4, 5, 6, 7]
available gates/columns = [1]
--------------------------------------------------
ITERATION-3
unique row sums = {2: 1, 3: 0, 4: 0, 5: 0, 6: 1, 7: 0}
deletable row = 2
deletable columns = [1]
Already Deleted columns = [0, 4, 5, 2, 3, 1]
available inputs/rows = [3, 4, 5, 6, 7]
[ NO AVAILABLE GATEs / COLUMNs ]
--------------------------

# **Display Fault Localization Table**

# **Fault Localization for SMGF & PMGF**

In [ ]:
# SMGF FAULT LOCALIZATION TABLE
print("\033[1;33;40m-------------------------------------------------SMGF-------------------------------------------------")
print(f"\n\033[1;37;42mOptimal Test Set = {optimal_test_set_smgf}\n")
fault_table_columns_smgf=(["INPUT","FAULT_LOCATION","ORIGINAL_OUTPUT","FAULTY_OUTPUT","INCREASING ORDER"])
df_fault_table=pd.DataFrame(fault_localization_table_smgf, columns=fault_table_columns_smgf)
df_fault_table
print("\033[1;37;41mNOTE : THE INCREASING ORDER OF THE FAULTS MUST BE FOLLOW FOR LOCALIZATION-------------------")


# PMGF FAULT LOCALIZATION TABLE
print("\n\n\n\n")
print("\033[1;33;40m-------------------------------------------------PMGF-------------------------------------------------")
print(f"\n\033[1;37;42mOptimal Test Set = {optimal_test_set_pmgf}\n")
fault_table_columns_pmgf=(["INPUT","PMGF_NAME","FAULT_LOCATION","ORIGINAL_OUTPUT","FAULTY_OUTPUT","INCREASING ORDER"])
df_fault_table=pd.DataFrame(fault_localization_table_pmgf, columns=fault_table_columns_pmgf)
df_fault_table
print("\033[1;37;41mNOTE : THE INCREASING ORDER OF THE FAULTS MUST BE FOLLOW FOR LOCALIZATION-------------------")

-------------------------------------------------SMGF-------------------------------------------------

Optimal Test Set = [['0', '1', '1'], ['0', '1', '0']]



,INPUT,FAULT_LOCATION,ORIGINAL_OUTPUT,FAULTY_OUTPUT,INCREASING ORDER
0,"[0, 1, 1]",GATE-1,"[1, 0, 0]","[0, 1, 1]",GATE-1
1,"[0, 1, 1]",GATE-2,"[1, 0, 0]","[1, 0, 1]",GATE-2
2,"[0, 1, 1]",GATE-4,"[1, 0, 0]","[1, 1, 1]",GATE-4
3,"[0, 1, 0]",GATE-3,"[0, 0, 1]","[0, 1, 0]",GATE-3
4,"[0, 1, 0]",GATE-5,"[0, 0, 1]","[0, 1, 1]",GATE-5


NOTE : THE INCREASING ORDER OF THE FAULTS MUST BE FOLLOW FOR LOCALIZATION-------------------





-------------------------------------------------PMGF-------------------------------------------------

Optimal Test Set = [['0', '0', '1'], ['0', '0', '0'], ['0', '1', '0']]



,INPUT,PMGF_NAME,FAULT_LOCATION,ORIGINAL_OUTPUT,FAULTY_OUTPUT,INCREASING ORDER
0,"[0, 0, 1]",PMGF-1,GATE=1 || INPUT-LINE=2,"[0, 1, 0]","[1, 0, 1]",PMGF-1
1,"[0, 0, 1]",PMGF-5,GATE=4 || INPUT-LINE=1,"[0, 1, 0]","[0, 0, 1]",PMGF-5
2,"[0, 0, 1]",PMGF-6,GATE=5 || INPUT-LINE=3,"[0, 1, 0]","[0, 0, 0]",PMGF-6
3,"[0, 0, 0]",PMGF-3,GATE=2 || INPUT-LINE=3,"[0, 0, 0]","[0, 0, 1]",PMGF-3
4,"[0, 0, 0]",PMGF-4,GATE=3 || INPUT-LINE=2,"[0, 0, 0]","[0, 1, 1]",PMGF-5 -> PMGF-4
5,"[0, 1, 0]",PMGF-2,GATE=1 || INPUT-LINE=3,"[0, 0, 1]","[1, 1, 0]",PMGF-2


NOTE : THE INCREASING ORDER OF THE FAULTS MUST BE FOLLOW FOR LOCALIZATION-------------------
